# CSCI_544 - HW3

In [1]:
import pandas as pd
import numpy as np
import random
import re
import os,sys
from collections import Counter
import nltk
#^[0-9]\d*([\.\,\-\\\/]*\d+)?$
#^[1-9]\d*(\.\d+)?$
regex = re.compile('^[0-9]\d*([\.\,\-\\\/]*\d+)?$')

In [2]:
f=open(os.path.join(sys.path[0],'train'),"r")
lines=f.read().splitlines()
#lines = f.readlines().rstrip('\n')
result= []

for x in lines:
    y = x.split('\t')
    #print(x.split('\t'))
    if len(y)==3:
        
        result.append(x.split('\t')[1].lower())
f.close()
#print(result)
print(len(result))

912095


In [3]:
from collections import Counter
my_dict ={}

my_dict = Counter(result)
#my_dict.update(my_count)

print(len(my_dict))

38558


In [4]:
n_cnt = 0
unk_val = 0
num_val = []
unk_val_lt = []
my_vocab = dict(sorted(my_dict.items(), key=lambda item: item[1],reverse=True))
print(len(my_vocab))
for i in list(my_vocab.keys()):
    #print(my_test[i])
    if regex.search(i)!=None:
        n_cnt+=1
        num_val.append(i)
        del my_vocab[i]
    elif my_vocab[i] < 2:
        unk_val_lt.append(i)
        del my_vocab[i]
        unk_val+=1
print(unk_val)
print(n_cnt)

print(len(my_vocab))
#print(my_vocab)

38558
14775
4293
19490


In [5]:
idx = 2
with open(os.path.join(sys.path[0],'vocab.txt'), 'w') as f:
    f.write('<unk>'+'\t'+str(0)+'\t'+str(unk_val)+'\n')
    f.write('<num>'+'\t'+str(1)+'\t'+str(n_cnt)+'\n')
    for i in my_vocab:
        #my_str = 
        f.write(str(i)+'\t'+str(idx)+'\t'+str(my_vocab[i])+'\n')
        idx+=1
    my_vocab['<unk>'] = unk_val
    my_vocab['<num>'] = n_cnt
f.close()

In [6]:
def preprocess(file_name):
    f=open(file_name,"r")
    lines=f.read().splitlines()
    my_tags_lt = []
    pos_tags= []
    my_pos_lt = []
    st_cnt=0
    temp = ['<s>']
    i = 0
    for x in lines:
        y = x.split('\t')
        if len(y) == 1:
            st_cnt+=1
            temp = ['<s>']
            i = 0
        else:
            temp.append(y[2])
            my_pos_lt.append((temp[i],y[2]))
            i+=1

        if len(y)==3:
            given_word = x.split('\t')[1].lower()
            
            if given_word in my_vocab:
                my_tags_lt.append((given_word,x.split('\t')[2]))
            elif regex.search(given_word)!=None:
                my_tags_lt.append(('<num>',x.split('\t')[2]))
            else:
                my_tags_lt.append(('<unk>',x.split('\t')[2]))

            pos_tags.append(x.split('\t')[2])
    f.close()
    my_count_pos = Counter(pos_tags)
    my_count_pos['<s>'] = st_cnt
    count_sx_pair = Counter(my_tags_lt)
    count_ss_pair = Counter(my_pos_lt)
    return my_count_pos,count_sx_pair,count_ss_pair,pos_tags

In [7]:
my_count_pos,count_sx_pair,count_ss_pair,pos_tags = preprocess("train")

In [8]:
print(set(pos_tags))

{"''", 'RBR', 'RB', 'WP$', 'PRP', 'IN', ':', 'JJR', 'TO', 'JJS', 'LS', 'RBS', 'VBG', 'NN', 'NNP', '$', '#', '.', 'JJ', 'WRB', 'SYM', 'DT', 'CD', 'EX', 'FW', 'VBZ', 'NNS', 'VBD', 'VB', 'MD', 'CC', 'WDT', 'VBP', '-LRB-', 'NNPS', '-RRB-', ',', 'POS', '``', 'UH', 'WP', 'VBN', 'PDT', 'PRP$', 'RP'}


In [9]:
pos_tags = set(pos_tags)

In [10]:
print(len(count_sx_pair))

27998


In [11]:
def emission_prob(count_sx_pair):
    em_dt = {}
    count_sx_pair_cnt = [(*i, j) for i, j in count_sx_pair.items()]
    #print(len(count_sx_pair_cnt))
    for i in count_sx_pair_cnt:
        count_sx = i[2]
        count_s = my_count_pos[i[1]]
        em_dt[(i[1],i[0])] = count_sx/count_s
    return em_dt


In [12]:
em_dt_rt = emission_prob(count_sx_pair)

In [13]:
# em_dt contains the emission dictionary 
print(len(em_dt_rt))

27998


In [14]:
def trans_prob(count_ss_pair):
    count_ss_pair_cnt = [(*i, j) for i, j in count_ss_pair.items()]
    trans_dt = {}
    for i in count_ss_pair_cnt:
        count_ss = i[2]
        count_s = my_count_pos[i[0]]
        trans_dt[(i[0],i[1])] = count_ss/count_s
    return trans_dt


In [15]:
trans_dt_rt = trans_prob(count_ss_pair)

In [16]:
print(len(trans_dt_rt))

1392


In [17]:
a = {str(k): trans_dt_rt[k] for k in trans_dt_rt} 
b = {str(k): em_dt_rt[k] for k in em_dt_rt} 

In [18]:
import json
task2_dict = {}
task2_dict['transition'] = a
task2_dict['emission'] = b
#print(task2_dict)
with open(os.path.join(sys.path[0],'hmm.json'), 'w') as f:
    json.dump(task2_dict,f)


In [19]:
def my_dev(file_name):
    f=open(file_name,"r")
    lines=f.read().splitlines()
    all_words = ['<s>']
    all_tags = []
    for x in lines:
        y = x.split('\t')
        if len(y) == 1:
            all_words.append('<e>')
            all_words.append('<s>')
        else:
            all_words.append(y[1].lower())
            all_tags.append(y[2])
    return all_words,all_tags
    

In [20]:
all_words,all_tags = my_dev(os.path.join(sys.path[0],'dev'))
all_words = all_words[:-1]


In [21]:
print(len(all_words),len(all_tags))

142820 131768


In [22]:
def find_trans_prob(t1,t2):
    if (t1,t2) in count_ss_pair:
        cnt_trans_ss = count_ss_pair[(t1,t2)]
        cnt_trans_s = my_count_pos[t1]
        #print('t1,t2',t1,t2,cnt_trans_ss,cnt_trans_s)
        return cnt_trans_ss/cnt_trans_s
    else:
        return 0


In [23]:
print(len(my_vocab))

19492


In [24]:
def find_ems_prob(t1,t2):
    if (t1,t2) in count_sx_pair:
        cnt_ems_ss = count_sx_pair[(t1,t2)]
        cnt_ems_s = my_count_pos[t2]
        #print('t1,t2',t1,t2,cnt_ems_ss,cnt_ems_s)
        try:
            return cnt_ems_ss/cnt_ems_s
        except:
            return 0
    elif t1 not in my_vocab :
    
        if regex.search(t1)!=None:
            cnt_ems_ss = count_sx_pair[('<num>',t2)]
            cnt_ems_s = my_count_pos[t2]
            #print('t1,t2',t1,t2,cnt_ems_ss,cnt_ems_s)
            try:
                return cnt_ems_ss/cnt_ems_s
            except:
                return 0
        else:
            cnt_ems_ss = count_sx_pair[('<unk>',t2)]
            cnt_ems_s = my_count_pos[t2]
            #print('t1,t2',t1,t2,cnt_ems_ss,cnt_ems_s)
            try:
                return cnt_ems_ss/cnt_ems_s
            except:
                return 0
    else:
        return 0

In [25]:
def Greedy(all_words,pos_tags):
    state = []
    T = list(pos_tags)
    #print('T',T)
    for i in range(1,len(all_words)):
        p = []
        #print('i',all_words[i])
        for tag in T:
            #print('tag',tag)
            if all_words[i] =='<e>':
                state.append('<e>')
                break
            elif all_words[i-1]=='<e>' and all_words[i] =='<s>':
                break
            elif all_words[i-1] =='<s>':
                transition_p = find_trans_prob(all_words[i-1],tag)
                #print('trnas start',transition_p)
            
            else:
                transition_p = find_trans_prob(state[-1],tag)
                #print('trans',transition_p)
            #if i!='':

            emission_p = find_ems_prob(all_words[i],tag)
            #print('emiss',emission_p)
            state_probability = emission_p * transition_p 
            #print('state prob',state_probability)   
            p.append(state_probability)
            #print('p',p)
        if len(p) == 0:
            pass
        else:
            pmax = max(p)
            state_max = T[p.index(pmax)]
            
            
            state.append(state_max)
    
    return list(state)

In [26]:
ans = Greedy(all_words,pos_tags)

In [27]:
to_acc = ans
ans = [i for i in ans if i != '<e>']


In [28]:
print(len(ans))
print(len(all_tags))
#print(ans)
#print((all_tags))
ans.append('.')
#print(ans)
print(len(ans))


131767
131768
131768


In [29]:
correct = 0
for i in range(len(ans)):
    
    if all_tags[i] == ans[i]:
        correct+=1
        
        

print(correct)
#print(we)
print('Accuracy:',correct/len(ans)*100)


121535
Accuracy: 92.23407807661951


In [30]:
def my_test(file_name):
    f=open(file_name,"r")
    lines=f.read().splitlines()
    all_words = ['<s>']
    #all_tags = []
    for x in lines:
        y = x.split('\t')
        if len(y) == 1:
            all_words.append('<e>')
            all_words.append('<s>')
        else:
            all_words.append(y[1].lower())
            #all_tags.append(y[2])
    return all_words
    

In [31]:
all_words= my_test(os.path.join(sys.path[0],'test'))
all_words = all_words[:-1]


In [32]:
ans_test = Greedy(all_words,pos_tags)
ans_test.append('.')

In [33]:
print(len(ans_test))

135115


In [34]:
f=open(os.path.join(sys.path[0],'test'),"r")
x = f
lines = x.read().splitlines()
print(len(lines))
#print(lines)
with open(os.path.join(sys.path[0],'greedy.out.txt'), 'w') as my_op:
    for i in range(len(lines)):
        if ans_test[i]!='<e>':
            mt = '\t'+ans_test[i]+'\n'
            #print(lines[i]+mt)
            #lines[i]=lines[i]+mt
            my_op.write(lines[i]+mt)
        else:
            my_op.write('\n')
        


135115


In [41]:
all_words = my_test(os.path.join(sys.path[0],'test'))
all_words = all_words[:-1]


In [42]:
def Viterbi(all_words,pos_tags):
    state = []
    T = list(pos_tags)
    #print('T',T)
    state_prob = []
    
    for i in range(1,len(all_words)):
        p = []
        flag = False
        chw = False
        print('i',all_words[i])
        for tag in T:
            print('tag',tag)
            if all_words[i] =='<e>':
                break
            elif all_words[i-1]=='<e>' and all_words[i] =='<s>':
                break
            elif all_words[i-1] =='<s>':
                transition_p = find_trans_prob(all_words[i-1],tag)
                print('trnas start',transition_p)
            
            else:
                print('in DT',state[-1][0])
                if len(state[-1][0])==2:
                    flag = True
                    dr = [state[-1][0][0]]
                    print('dr')
                    transition_p = find_trans_prob(state[-1][0][0],tag)
                    print(transition_p)
                else:
                    print('in chw')
                    chw = True
                    dr = [state[-1][0][0]]
                    print(dr)
                    saq = {}

                    for i in range(cnt):
                        transition_p = find_trans_prob(state[-1-i][0][-1],tag)
                        saq[state[-1-i][0]] = transition_p*state[-1-i][1]
                    print('saq,cnt',cnt,saq)
                #print('trans',transition_p)
            #if i!='':

            emission_p = find_ems_prob(all_words[i],tag)
            #print('emiss',emission_p)
            if chw ==True:
                er = {}
                for q in saq:
                    er[q] = emission_p * saq[q]
                max_key = max(er, key=er.get)
                state_probability = er[max_key]
            else:
                state_probability = emission_p * transition_p 
            #print('state prob',state_probability)   
            if flag == True:
                p.append(state_probability*state[-1][1])
            elif chw ==True:
                p.append(state_probability)
            else:
                p.append(state_probability)
            #print('p',p)
        if len(p) == 0:
            pass
        else:
            v_temp = []
            cnt = 0
            for i in p:
                if p!=0 and flag == False:
                    v_temp.append(T[p.index(i)])
                    state.append([v_temp,p])
                elif p!=0 and flag==True:
                    cnt+=1
                    dr.append(T[p.index(i)])
                    v_temp.append(dr)
                    state.append([v_temp,p])
                elif p!=0 and chw==True:
                    cnt+=1
                    dr.append(T[p.index(i)])
                    v_temp.append(dr)
                    state.append([v_temp,p])

                
    
    
    return list(state)

In [37]:
ans = Viterbi(all_words,pos_tags)

i the
tag ''
trnas start 0.000366329120548447
tag RBR
trnas start 0.0020933092602768402
tag RB
trnas start 0.056048355443912394
tag WP$
trnas start 2.61663657534605e-05
tag PRP
trnas start 0.06149095952063218
tag IN
trnas start 0.12884318497003952
tag :
trnas start 0.002799801135620274
tag JJR
trnas start 0.0017008137739749326
tag TO
trnas start 0.003532459376717168
tag JJS
trnas start 0.0024858047465787475
tag LS
trnas start 0.0009158228013711175
tag RBS
trnas start 0.0005233273150692101
tag VBG
trnas start 0.01201036188083837
tag NN
trnas start 0.04118585969594683
tag NNP
trnas start 0.1978962241934218
tag $
trnas start 0.0008634900698641965
tag #
trnas start 2.61663657534605e-05
tag .
trnas start 0
tag JJ
trnas start 0.04170918701101604
tag WRB
trnas start 0.006096763220556297
tag SYM
trnas start 0.0010989873616453411
tag DT
trnas start 0.21911714681947825
tag CD
trnas start 0.011225370908234555
tag EX
trnas start 0.004238951252060601
tag FW
trnas start 0.0001831645602742235
tag VBZ

ValueError: max() arg is an empty sequence